In [43]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# == 创建 EE 原始图形 ==
theta1 = np.linspace(np.pi, 110 * np.pi / 180, int((np.pi - 110 * np.pi / 180) / 0.01) + 1)
theta2 = np.linspace(60 * np.pi / 180, np.pi, int((np.pi - 60 * np.pi / 180) / 0.01) + 1)
L4 = 0.385

EEside = np.concatenate([
    31.75 * np.exp(1j * theta1),
    [-10.86 + 29.83j,
     105.29 + 72.17j,
     119.68 + 80.76j,
     129.5 + 80.76j,
     129.5 + 96j,
     55 + 96j,
     -148 + 50j,
     -160 + 44.7j],
    -L4 * 1000 + 49 * np.exp(1j * theta2)
]) / 1000

EE_full = L4 + np.concatenate([EEside[::-1], np.real(EEside) - 1j * np.imag(EEside)])
wafer_radius = 0.15
wafer_base = wafer_radius * np.exp(1j * np.linspace(0, 2*np.pi, 200)) + 0.385

# == 边界绘图 ==
def draw_borders(ax):
    x1 = np.linspace(0, 2, 100);     y1 = np.ones_like(x1) * 1.075
    x2 = np.linspace(0, 0.8, 100);   y2 = np.ones_like(x2) * 0.5
    x9 = np.linspace(1.215, 2, 100); y9 = np.ones_like(x9) * 0.5
    y3 = np.linspace(0.5, 0.4, 100); x3 = np.ones_like(y3) * 0.8
    y8 = np.linspace(0.4, 0.5, 100); x8 = np.ones_like(y8) * 1.215
    x4 = np.linspace(0.8, 0.83, 100); y4 = np.ones_like(x4) * 0.4
    x7 = np.linspace(1.185, 1.215, 100); y7 = np.ones_like(x7) * 0.4
    y5 = np.linspace(0.4, 0, 100); x5 = np.ones_like(y5) * 0.83
    y6 = np.linspace(0, 0.4, 100); x6 = np.ones_like(y6) * 1.185
    segments = [(x1, y1), (x2, y2), (x3, y3), (x4, y4), (x5, y5),
                (x6, y6), (x7, y7), (x8, y8), (x9, y9)]
    for x, y in segments:
        ax.plot(x, y, 'r', linewidth=1.5)

# == 控制图更新 ==
def update_plot(x_shift=0.1, y_shift=0.1, theta_deg=0):
    theta = np.deg2rad(theta_deg)
    center = x_shift + 1j * y_shift
    EE_trans = (EE_full - 0.385) * np.exp(1j * theta) + center
    wafer_trans = (wafer_base - 0.385) * np.exp(1j * theta) + center

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, 2)
    ax.set_ylim(0, 1.2)
    ax.set_aspect('equal')
    draw_borders(ax)
    ax.fill(EE_trans.real, EE_trans.imag, color='gray', alpha=0.6, label='End Effector')
    ax.fill(wafer_trans.real, wafer_trans.imag, color='red', alpha=0.4, label='Wafer')
    ax.set_title(f"EE Center: ({x_shift:.3f}, {y_shift:.3f}) | θ = {theta_deg}°")
    ax.legend()
    plt.grid(True)
    plt.show()

# == 启动交互滑块 ==
interact(
    update_plot,
    x_shift=FloatSlider(value=0.1, min=0, max=2, step=0.01, description='X位置'),
    y_shift=FloatSlider(value=0.1, min=0, max=1.2, step=0.01, description='Y位置'),
    theta_deg=FloatSlider(value=0, min=0, max=360, step=1, description='旋转角')
);


interactive(children=(FloatSlider(value=0.1, description='X位置', max=2.0, step=0.01), FloatSlider(value=0.1, de…

In [44]:
import numpy as np

class EEState:
    def __init__(self, x, y, theta_deg):
        self.x = x
        self.y = y
        self.theta_deg = theta_deg
        self.center = x + 1j * y

    def transform(self, EE_full):
        """返回当前状态下的 EE 形状（复数数组）"""
        theta_rad = np.deg2rad(self.theta_deg)
        return (EE_full - 0.385) * np.exp(1j * theta_rad) + self.center

    def is_valid(self, EE_full, bounds_func):
        """检查该状态下 EE 是否越界"""
        ee_shape = self.transform(EE_full)
        return bounds_func(ee_shape)

    def distance_to(self, other_state):
        """状态之间的欧几里得距离 + 角度差作为 cost"""
        dx = self.x - other_state.x
        dy = self.y - other_state.y
        dtheta = (self.theta_deg - other_state.theta_deg + 180) % 360 - 180
        return np.sqrt(dx**2 + dy**2) + abs(dtheta) / 180  # 角度也参与代价


In [45]:
import random

class RRTNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent

def sample_random_state(x_range, y_range, theta_range=(0, 360), goal_state=None, goal_sample_rate=0.2):
    """引导采样：有一定概率直接采样目标点"""
    if goal_state and random.random() < goal_sample_rate:
        return goal_state
    x = random.uniform(*x_range)
    y = random.uniform(*y_range)
    theta = random.uniform(*theta_range)
    return EEState(x, y, theta)

def find_nearest_node(tree, new_state):
    min_dist = float('inf')
    nearest = None
    for node in tree:
        d = node.state.distance_to(new_state)
        if d < min_dist:
            min_dist = d
            nearest = node
    return nearest

def steer_towards(from_state, to_state, step_size=0.05, max_angle_step=10):
    dx = to_state.x - from_state.x
    dy = to_state.y - from_state.y
    dtheta = (to_state.theta_deg - from_state.theta_deg + 180) % 360 - 180

    dist = np.sqrt(dx**2 + dy**2)
    if dist > step_size:
        ratio = step_size / dist
        dx *= ratio
        dy *= ratio
    if abs(dtheta) > max_angle_step:
        dtheta = np.sign(dtheta) * max_angle_step

    new_x = from_state.x + dx
    new_y = from_state.y + dy
    new_theta = from_state.theta_deg + dtheta
    return EEState(new_x, new_y, new_theta)


In [46]:
import matplotlib.pyplot as plt

def rrt_plan(start_state, goal_state, EE_full, bounds_func, max_iter=1000):
    tree = [RRTNode(start_state)]
    goal_tolerance = 0.1

    for _ in range(max_iter):
        rand_state = sample_random_state((0.1, 1.9), (0.1, 1.1))
        nearest = find_nearest_node(tree, rand_state)
        new_state = steer_towards(nearest.state, rand_state)

        if not new_state.is_valid(EE_full, bounds_func):
            continue

        new_node = RRTNode(new_state, nearest)
        tree.append(new_node)

        # Check if close enough to goal
        if new_state.distance_to(goal_state) < goal_tolerance:
            goal_node = RRTNode(goal_state, new_node)
            tree.append(goal_node)
            return goal_node, tree

    return None, tree

def extract_path(goal_node):
    path = []
    node = goal_node
    while node:
        path.append(node.state)
        node = node.parent
    path.reverse()
    return path

def plot_rrt_path(path, EE_full, draw_borders):
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, 2)
    ax.set_ylim(0, 1.2)
    ax.set_aspect('equal')
    draw_borders(ax)
    for state in path:
        ee = state.transform(EE_full)
        ax.plot(ee.real, ee.imag, alpha=0.4)
        ax.plot(state.x, state.y, 'bo', markersize=2)
    ax.set_title("Planned Path")
    plt.grid(True)
    plt.show()


In [47]:
start = EEState(1.0, 0.15, 270)
goal = EEState(1.8, 0.9, 60)

goal_node, tree = rrt_plan(start, goal, EE_full, is_within_custom_bounds, max_iter=2000)

if goal_node:
    path = extract_path(goal_node)
    plot_rrt_path(path, EE_full, draw_borders)
else:
    print("❌ Failed to find a path.")


❌ Failed to find a path.


In [48]:
def is_within_custom_bounds(ee_shape):
    """
    Checks if the End Effector shape (complex numbers) is within the defined custom bounds.
    The bounds are defined by the border segments in draw_borders.
    """
    # Define the boundaries
    # These correspond to the segments defined in draw_borders
    x_bounds = [0, 2, 0, 0.8, 0.8, 0.83, 0.83, 1.185, 1.185, 1.215, 1.215, 2, 2]
    y_bounds = [0, 0, 0.5, 0.5, 0.4, 0.4, 0, 0, 0.4, 0.4, 0.5, 0.5, 1.075]

    # Check if all points of the EE are within the overall rectangular boundary [0, 2] x [0, 1.075]
    if not (np.all(ee_shape.real >= 0) and np.all(ee_shape.real <= 2) and
            np.all(ee_shape.imag >= 0) and np.all(ee_shape.imag <= 1.075)):
        return False

    # Check against the inner boundaries (the cutout area)
    # A point is outside the cutout if it's within the rectangle [0.8, 1.215] x [0, 0.5]
    # but outside the smaller rectangle [0.83, 1.185] x [0, 0.4]
    cutout_area1 = (ee_shape.real >= 0.8) & (ee_shape.real <= 1.215) & (ee_shape.imag >= 0) & (ee_shape.imag <= 0.5)
    cutout_area2 = (ee_shape.real >= 0.83) & (ee_shape.real <= 1.185) & (ee_shape.imag >= 0) & (ee_shape.imag <= 0.4)

    # If any point is in cutout_area1 but not in cutout_area2, it's in the invalid region
    if np.any(cutout_area1 & ~cutout_area2):
        return False

    return True

In [49]:
start = EEState(1.0, 0.15, 270)
goal = EEState(1.8, 0.9, 60)

if not goal.is_valid(EE_full, is_within_custom_bounds):
    print("❌ Goal state is invalid!")

goal_node, tree = rrt_plan(start, goal, EE_full, is_within_custom_bounds, max_iter=10000)

if goal_node:
    path = extract_path(goal_node)
    plot_rrt_path(path, EE_full, draw_borders)
else:
    print("❌ Failed to find a path.")


❌ Failed to find a path.
